## Code for reproducing analysis and figure in paper

**IMPORTANT:** Note all code in this folder is only meant to be used for reproducing analysis in the paper, not for analyzing your own output as there are hard-coded parameters that only makes sense for specific datasets

In [ ]:
import morpheus as mp

#optional: set the default style for matplotlib
!pip install umap-learn mplscience statsmodels -Uqqqq
import mplscience
mplscience.available_styles()
mplscience.set_style(reset_current=True)

%reload_ext autoreload
%autoreload 2

### Load data sets

In [ ]:
# initialize a SpatialDataset Object (modify the path to the data on your machine)
BASE_DIR = "/groups/mthomson/zwang2/IMC/paper/melanoma"
mla_data = mp.SpatialDataset(
    input_path=f"{BASE_DIR}/singlecell.csv",  # path to the single cell data
    cf_dir=f"{BASE_DIR}/cf/run_0",  # path to the computed perturbations
    model_path=f"{BASE_DIR}/model/unet.ckpt",  # path to the trained model
    additional_cols=["Cancer_Stage"],
)

BASE_DIR = "/groups/mthomson/zwang2/IMC/paper/crc"
crc_data = mp.SpatialDataset(
    input_path=f"{BASE_DIR}/singlecell.csv",
    cf_dir=f"{BASE_DIR}/cf/run_0",
    model_path=f"{BASE_DIR}/model/unet.ckpt",
    additional_cols=["type", "FLD"],
)

BASE_DIR = "/groups/mthomson/zwang2/IMC/paper/breast"
bst_data = mp.SpatialDataset(
    input_path=f"{BASE_DIR}/singlecell.csv",
)

### 0. Assess model performance

In [ ]:
from analysis_helper import get_rmse_and_prediction, optimize_threshold
from figure_helper import plot_prediction_scatterplot, plot_rmse

In [ ]:
all_pred = {}
all_rmse = {}
thresholds = {}
for name, data in {
    "Melanoma": mla_data,
    "CRC": crc_data,
    "Breast tumor": bst_data,
}.items():
    thresh = optimize_threshold(data, split="validate")
    pred, rmse = get_rmse_and_prediction(data, split="test", classify_threshold=thresh)
    print(f"{name} RMSE: {rmse}")
    all_pred[name] = pred
    all_rmse[name] = rmse
    thresholds[name] = thresh

plot_prediction_scatterplot(all_pred)
plot_rmse(all_rmse)

### 1. Visualizing patient-level perturbations

In [ ]:
# import relevant functions
from analysis_helper import retrieve_perturbation
from figure_helper import plot_patient_perturbation

##### Melanoma

In [ ]:
cf_mla, channel_to_perturb_mla = retrieve_perturbation(
    mla_data, additional_col=["Cancer_Stage"]
)
strategy_mla, patient_cf_mla = plot_patient_perturbation(
    mla_data,
    cf_mla,
    channel_to_perturb_mla,
    numClust=3,
    extra_cols=["Cancer_Stage"],
    keep_top_2=True,
)
display(strategy_mla)

##### CRC

In [ ]:
cf_crc, channel_to_perturb_crc = retrieve_perturbation(
    crc_data, additional_col=["type", "FLD"]
)
strategy_crc, patient_cf_crc = plot_patient_perturbation(
    crc_data,
    cf_crc,
    channel_to_perturb_crc,
    numClust=2,
    extra_cols=["FLD"],
)
display(strategy_crc)

### 2. Assess effectiveness of perturbation on test cohort

In [ ]:
# import relevant functions
from analysis_helper import assess_perturbation
from figure_helper import plot_perturbation_performance

##### Melanoma

In [ ]:
tcell_level_melanoma = assess_perturbation(
    mla_data,
    strategy_mla,
    data_split="test",
    classify_threshold=thresholds["Melanoma"],
    additional_col=["Cancer_Stage"],
)

plot_perturbation_performance(
    tcell_level_melanoma,
)

##### CRC

In [ ]:
tcell_level_crc = assess_perturbation(
    crc_data,
    strategy_crc,
    data_split="test",
    classify_threshold=thresholds["CRC"],
    additional_col=["FLD", "type"],
)

tcell_level_crc["FLD"] = tcell_level_crc.apply(
    lambda x: 1 if x["FLD"] > 0 else 0, axis=1
)

plot_perturbation_performance(
    tcell_level_crc,
    patient_phenotype="FLD",
    strategy_mapping={"strategy_1": 1, "strategy_2": 0},
)

### 3. Visualizing perturbation vectors in UMAP space

In [ ]:
from analysis_helper import get_umap_embeddings
from figure_helper import plot_umap_embedding, plot_umap_embedding_crc

In [ ]:
embedding_mla, umap_cf_mla = get_umap_embeddings(
    mla_data, cf_mla, channel_to_perturb_mla
)

In [ ]:
plot_umap_embedding(embedding_mla, umap_cf_mla, pie_chart=True, save_fig=True)

In [ ]:
embedding_crc, umap_cf_crc = get_umap_embeddings(
    crc_data, cf_crc, channel_to_perturb_crc, additional_col=["type"]
)
plot_umap_embedding_crc(embedding_crc, umap_cf_crc)

### 4. Comparing patients across clusters with volcano plots

In [ ]:
from analysis_helper import differential_analysis_celltype, differential_analysis_genes
from figure_helper import make_volcano_plot

In [ ]:
mla_celltype_da = differential_analysis_celltype(mla_data, patient_cf_mla)
mla_genes_da = differential_analysis_genes(
    mla_data, patient_cf_mla, channel_to_perturb_mla
)
make_volcano_plot(mla_genes_da, mla_celltype_da)

In [ ]:
crc_celltype_da = differential_analysis_celltype(crc_data, patient_cf_crc)
crc_genes_da = differential_analysis_genes(
    crc_data, patient_cf_crc, channel_to_perturb_crc
)

In [ ]:
make_volcano_plot(crc_genes_da, crc_celltype_da, save_fig=True)

### 5. Tissue-level perturbation heatmap (CRC only)

In [ ]:
from figure_helper import plot_tissue_level_perturbation

In [ ]:
plot_tissue_level_perturbation(cf_crc, channel_to_perturb_crc, save_fig=True)

### 6. Patch-level perturbation heatmap (Melanoma only)

In [ ]:
from analysis_helper import get_IHC_subset
from figure_helper import make_patch_heatmap_mla

In [ ]:
X, label = get_IHC_subset(mla_data, channel_to_perturb_mla)
make_patch_heatmap_mla(X, label, save_fig=True)

### 7. strategy at different sparsity

In [ ]:
from analysis_helper import assess_perturbation
from figure_helper import make_multi_strat_plot
import pandas as pd

In [ ]:
allStrategy = pd.read_csv("allstrat_crc.csv")
tcell_crc = assess_perturbation(
    crc_data,
    allStrategy,
    data_split="test",
    classify_threshold=0.33,  # 0.33 for CRC
    additional_col=["FLD", "type"],
)
infiltrate_lower, infiltrate_upper = make_multi_strat_plot(
    allStrategy,
    tcell_crc,
    chemlabel=["CXCR4", "PD1", "CYR61", "PDL1", "IL10"],
    save_fig=True,
)

In [ ]:
allStrategy = pd.read_csv("allstrat_mla.csv")
tcell_mla = assess_perturbation(
    mla_data,
    allStrategy,
    data_split="test",
    classify_threshold=0.31,  # 0.33 for melanoma
)
infiltrate_lower, infiltrate_upper = make_multi_strat_plot(
    allStrategy,
    tcell_mla,
    chemlabel=["CXCL9", "CCL18", "CCL22", "CXCL10", "CXCL12", "CXCL8"],
    save_fig=True,
)

### 8. Correlation plots

In [ ]:
from analysis_helper import get_feature_correlation
from figure_helper import plot_correlation

In [ ]:
corr_mla = get_feature_correlation(mla_data, channel_to_perturb_mla)
plot_correlation(corr_mla, save_fig=True)

In [ ]:
corr_crc = get_feature_correlation(crc_data, channel_to_perturb_crc)
plot_correlation(corr_crc, save_fig=True)

### 9. T cell map

In [ ]:
from figure_helper import get_example_Tcell_map, make_multiple_Tcell_map

In [ ]:
get_example_Tcell_map(mla_data, tcell_level_mla, img=23)

In [ ]:
make_multiple_Tcell_map(mla_data, 0.31, save_fig=True)

In [ ]:
make_multiple_Tcell_map(crc_data, 0.33, save_fig=True)

In [ ]:
make_multiple_Tcell_map(bst_data, 0.14, save_fig=True)